In [ ]:
"""Load packages"""
import numpy as np
import pandas as pd
import xarray as xr
import netCDF4 as nc
import scipy as sp
import matplotlib.pyplot as plt
import glob
import matplotlib.colors as colors
import uncertainties
from mpl_toolkits import Basemap
from math import radians, sin, cos, sqrt, atan2
from shapely.geometry import Polygon, Point
from scipy.optimize import curve_fit
from scipy.signal import argrelmax
from scipy.signal import find_peaks
from scipy.ndimage import gaussian_filter1d
from scipy.stats import linregress
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.transforms import Bbox

In [ ]:
"""Load locations latitude and longitude"""

moscow = np.array([36.75,55,38.75,57])
lipetskaya = np.array([38.5,52,40.25,53.5])
bagdad = np.array([43.75,32.5,45.5,34])
medupi_matimba = np.array([26.5,-24.5,27.75,-23.5])
australia_wildfires = np.array([131,-18,132.5,-16.5])

"""Load constants"""
qa_threshold = 0.75  # qa-value threshold
vminno2, vmaxno2 = 2e-5, 10e-5  # selected range of NO$_2$ values
vminco2, vmaxco2 = 400, 410  # selected range of CO$_2$ values
buffer_bounding_box = 0.1  # buffer for bounding box
data_points_width_k = 20  # width of number of data points in the satellite swath
path = '/home/gsl/Desktop/ArticleFun/'

In [ ]:
"""Load dataframe"""

df = pd.DataFrame([moscow, lipetskaya, bagdad, medupi_matimba, australia_wildfires], columns=['min_lon', 'min_lat', 'max_lon', 'max_lat'])
df.insert(0, 'name', ['Moscow', 'Lipetskaya', 'Bagdad', 'Medupi-Matimba', 'AustraliaWildFire'])

"""Calculate center of bounding box"""
df['center_lon'] = np.mean(np.array([df['min_lon'], df['max_lon']]), axis=0)
df['center_lat'] = np.mean(np.array([df['min_lat'], df['max_lat']]), axis=0)

"""Add paths to dataframe"""
df['path_s5p'] = pd.Series([glob.glob(path+df['name'][i]+f'/S5P_RPRO_L2__NO2____*.nc') for i in range(len(df))]).str[0]
df['path_oco2'] = pd.Series([glob.glob(path+df['name'][i]+f'/oco2_LtCO2_*.nc4') for i in range(len(df))]).str[0]
df['path_era5'] = pd.Series([glob.glob(path+df['name'][i]+f'/new_adaptor.mars.internal-*.nc') for i in range(len(df))]).str[0]
df['path_spread_era5'] = pd.Series([glob.glob(path+df['name'][i]+f'//spread_adaptor.mars.internal-*.nc') for i in range(len(df))]).str[0]
df

In [ ]:
def load_s5p_data(s5pfiles, min_lat, max_lat, min_lon, max_lon):
    """Load S5P data and return latitude, longitude, NO2, sigma_NO2, time, and NO2 units.
    The data is filtered by the selected area and qa-value threshold.
    """
    s5pdataset = nc.Dataset(s5pfiles, 'r')
    time = pd.to_datetime(s5pdataset.groups['PRODUCT'].variables['time_utc'][:][0], utc=True, format='ISO8601')  # [0] gets rid of extra dimension
    lat = s5pdataset.groups['PRODUCT'].variables['latitude'][:]
    lon = s5pdataset.groups['PRODUCT'].variables['longitude'][:]
    boolmask = (lat > min_lat) & (lat < max_lat) & (lon > min_lon) & (lon < max_lon)  # filter by selected area
    qa_value = s5pdataset.groups['PRODUCT'].variables['qa_value'][:][boolmask]  # filter by qa-value
    qa_mask = qa_value > qa_threshold
    lat = lat[boolmask][qa_mask]  # apply mask
    lon = lon[boolmask][qa_mask]  # apply mask
    time = time[np.argwhere(boolmask == True)[:,1]]
    no2 = s5pdataset.groups['PRODUCT'].variables['nitrogendioxide_tropospheric_column'][:][boolmask][qa_mask]
    sigma_no2 = s5pdataset.groups['PRODUCT'].variables['nitrogendioxide_tropospheric_column_precision'][:][boolmask][qa_mask]
    no2_units = s5pdataset.groups['PRODUCT'].variables['nitrogendioxide_tropospheric_column'].units
    return lat, lon, no2, sigma_no2, time, no2_units


def load_oco2_data(oco2files, min_lat, max_lat, min_lon, max_lon):
    """Load OCO2 data and return latitude, longitude, CO2, sigma_CO2, time, and CO2 units.
    The data is filtered by the selected area.
    """
    oco2dataset = nc.Dataset(oco2files, 'r')
    time = pd.to_datetime(oco2dataset.variables['time'][:], unit='s', utc=True)
    lat = oco2dataset.variables['latitude'][:]
    lon = oco2dataset.variables['longitude'][:]
    boolmask = (lat > min_lat) & (lat < max_lat) & (lon > min_lon) & (lon < max_lon) # filter by selected area
    lat = lat[boolmask]  # apply mask
    lon = lon[boolmask]
    time = time[boolmask]
    co2 = oco2dataset.variables['xco2'][:][boolmask]
    sigma_co2 = oco2dataset.variables['xco2_uncertainty'][:][boolmask]
    co2_units = oco2dataset.variables['xco2'].units
    return lat, lon, co2, sigma_co2, time, co2_units


def load_era5_data(era5files):
    """Load ERA5 data and return latitude, longitude, u10, v10, u100, v100, time, and tcc.
    """
    era5dataset = nc.Dataset(era5files, 'r')
    era5timeds = xr.open_dataset(era5files)
    time = pd.to_datetime(era5timeds.time.values, utc=True, format='ISO8601')
    lat = era5dataset.variables['latitude'][:]
    lon = era5dataset.variables['longitude'][:]
    u10 = era5dataset.variables['u10'][:]
    v10 = era5dataset.variables['v10'][:]
    u100 = era5dataset.variables['u100'][:]
    v100 = era5dataset.variables['v100'][:]
    tcc = era5dataset.variables['tcc'][:]
    return lat, lon, u10, v10, u100, v100, time, tcc

In [ ]:
def make_basemap(ax, latbox, lonbox, lat_0, lon_0, addlines=True,):
    """Make a basemap for the selected area.
    Possible to toggle on/off coastlines, countries, states, and map boundary.
    """
    m = Basemap(  # create Basemap with manually selcted view settings 
            width=250000,
            height=220000,
            resolution='l',
            projection='stere',
            lat_ts=40,
            lat_0=lat_0,
            lon_0=lon_0,
            ax=ax)
    if addlines:
        m.drawcoastlines()
        m.drawcountries()
        m.drawstates()
        m.drawmapboundary(fill_color='white')
        m.drawcoastlines(linewidth=0.5)
        m.drawcountries(linewidth=0.5)
        m.fillcontinents(color='white', lake_color='white')
    m.drawparallels(np.arange(np.floor(latbox[0]-2), np.floor(latbox[1]+2), 0.5), labels=[1, 0, 0, 0])
    m.drawmeridians(np.arange(np.floor(lonbox[0]-2), np.floor(lonbox[1]+2), 1), labels=[0, 0, 0, 1])
    return m

In [ ]:
def plotno2_func(ax, m, lon, lat, lat_origin, lon_origin, no2, no2_units, uwind_center, vwind_center, vminno2, vmaxno2, dot_label):
    """Plot NO2 data on a basemap."""
    x, y = m(lon, lat)
    x0, y0 = m(lon_origin, lat_origin)
    normalize = colors.Normalize(vmin=vminno2, vmax=vmaxno2)  # normalize NO2 values in selected range
    cs = m.scatter(x, y, c=np.squeeze(no2), norm=normalize, cmap='jet', s=25, alpha=0.5)
    ax.quiver(x0, y0, uwind_center, vwind_center, cmap='jet', scale=30, width=0.01, alpha=0.8, label=f'{dot_label}')  # plot wind vector
    cbar = m.colorbar(cs, location='right', pad="10%")
    cbar.set_label(r'NO$_2$ [$\frac{mol}{m^{2}}$]') 

def plotco2_func(ax, m, lon, lat, lat_origin, lon_origin, co2, co2_units, uwind_center, vwind_center, vminco2, vmaxco2, dot_label):
    """Plot CO2 data on a basemap."""
    x, y = m(lon, lat)
    x0, y0 = m(lon_origin, lat_origin)
    normalize = colors.Normalize(vmin=vminco2, vmax=vmaxco2)  # normalize CO2 values in selected range
    cs = m.scatter(x, y, c=np.squeeze(co2), norm=normalize, cmap='jet', s=25, alpha=0.5)
    ax.quiver(x0, y0, uwind_center, vwind_center, cmap='jet', scale=30, width=0.01, alpha=0.8, label=f'{dot_label}') # plot wind vector
    cbar = m.colorbar(cs, location='right', pad="10%")
    cbar.set_label(f'CO$_2$ [{co2_units}]')

def plotwind_func(ax, m, xwindmat, ywindmat, uwind, vwind, ws, label1):
    """Plot wind data field on a basemap."""
    xwindmat, ywindmat = np.meshgrid(xwindmat, ywindmat)
    mxwindmat, mywindmat = m(xwindmat, ywindmat)
    normalize = colors.Normalize(vmin=0, vmax=10)
    cf = ax.contourf(mxwindmat, mywindmat, ws, norm=normalize, cmap='jet')
    ax.quiver(mxwindmat, mywindmat, uwind, vwind, cmap='jet', scale=30, width=0.01, alpha=0.5, label=f'{label1}')
    cb = m.colorbar(cf, location='right', pad="10%")
    cb.set_label(r'$\frac{m}{s}$')

In [ ]:
def haversine(latlon1, latlon2):
    """Calculate the great circle distance between two points on the earth (specified in decimal degrees).
    """
    lat1, lon1, lat2, lon2 = latlon1[0], latlon1[1], latlon2[0], latlon2[1]
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    R = 6371.0088
    distance = R * c
    return distance

In [ ]:
def non_log_model_no2(x, a0, a1, a2, a3, a4):
    """Model for NO2 data without log transformation."""
    return a0 + a1*x + a2*np.exp(-4*np.log(2)*(x-a3)**2/(a4**2))

def non_log_model_co2(x, a5, a6, a7, a8, a4):
    """Model for CO2 data without log transformation."""
    return a5 + a6*x + a7*np.exp(-4*np.log(2)*(x-a8)**2/(a4**2))


def model_no2(x, a0, a1, a2, a3, la4):
    """Model for NO2 data with log transformation."""
    return a0 + a1*x + a2*np.exp(-4*np.log(2)*(x-a3)**2/(np.exp(la4)**2))

def model_co2(x, a5, a6, a7, a8, la4):
    """Model for CO2 data with log transformation."""
    return a5 + a6*x + a7*np.exp(-4*np.log(2)*(x-a8)**2/(np.exp(la4)**2))


"""The two functions below are used to fixate the a4 parameter for CO2 model."""
def model_co2_no_la4(x, a5, a6, a7, a8, la4):
    """Model for CO2 data with log transformation without la4. 
    la4 is fixated in the wrapper function.
    """
    return a5 + a6*x + a7*np.exp(-4*np.log(2)*(x-a8)**2/(np.exp(la4)**2))

def wrapperfunc(la4_fixed):
    """Wrapper function for model_co2_no_la4.
    This is to fixate the la4 parameter in the curve_fit function."""
    def tempfunc(x, a5, a6, a7, a8, la4=la4_fixed):
        return model_co2_no_la4(x, a5, a6, a7, a8, la4)
    return tempfunc

In [ ]:
def bounding_box(oco2lat, oco2lon, k_numbers=70, buffer = 0.0):
    """Calculate bounding box for the selected area."""
    lat1, lon1 = (np.max(oco2lat), np.min(oco2lon))  # Top left
    lat3, lon3 = (np.min(oco2lat), np.max(oco2lon))  # Bottom right
    top_k_lat_idx, bottom_k_lat_idx = np.argpartition(oco2lat, -k_numbers)[-k_numbers:], np.argpartition(oco2lat, k_numbers)[:k_numbers]
    lon2 = np.max(oco2lon[top_k_lat_idx])
    lon4 = np.min(oco2lon[bottom_k_lat_idx])
    lat2 = lat1  # Same y-position in the top
    lat4 = lat3  # Same y-position in the bottom
    return lon1-buffer, lat1, lon2+buffer, lat2, lon3+buffer, lat3, lon4-buffer, lat4

In [ ]:
def ini_guess(x, y):
    """Initial guess for curve_fit function."""
    smoothed_data = gaussian_filter1d(y, sigma=2)  # smooth data package
    peaks, _ = find_peaks(smoothed_data)  # find peaks on smoothed data
    A_guess = smoothed_data[peaks[np.argmax(smoothed_data[peaks])]]  # amplitude guess based on highest peak in smoothed data
    mu_guess = np.sum(x * y) / np.sum(y)  # mean guess based on weighted average
    sigma_guess = np.sqrt(np.sum(y * (x - mu_guess)**2) / np.sum(y))
    slope_guess, intercept_guess, _, _, _ = linregress(x, y)
    return np.array([intercept_guess, slope_guess, A_guess, mu_guess, np.log(sigma_guess)])

In [ ]:
def stringcalc(center_wind_speed_10m, sigma_ws10m, center_wind_speed_100m, sigma_ws100m):
    """Make string for wind speed plot."""
    str_10m_wind = uncertainties.ufloat(center_wind_speed_10m, sigma_ws10m)
    str_100m_wind = uncertainties.ufloat(center_wind_speed_100m, sigma_ws100m)
    return f'10m wind: {str_10m_wind:.1uf}', f'100m wind: {str_100m_wind:.1uf}'

In [ ]:
"""Set up plotting parameters"""

plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.size'] = 14
plt.rc('axes', labelsize=19)    # fontsize of the x and y labels - Specifically chosen for presentation

inches_to_cm = 2.54
scalar = 10
figsize_save = (scalar * 0.5 * 16/inches_to_cm,  scalar * (16/7 * 5) /inches_to_cm)

In [ ]:
"""Set up lists for saving data"""
origin_list = []

sin_theta_min_list10m = []
desired_length_list10m = []
final_vector_list10m = []
theta_min_deg_list10m = []

sin_theta_min_list100m = []
desired_length_list100m = []
final_vector_list100m = []
theta_min_deg_list100m = []

new_params_no2_list = []
new_params_co2_list = []
s5p_err_no2_list = []
s5p_err_co2_list = []
sigma_ws10m_list = []
sigma_ws100m_list = []
sigma_theta10m_list = []
sigma_theta100m_list = []


for i in range(len(df)):
    """Load data and calculate wind speed and wind direction."""
    s5plat, s5plon, s5pno2, s5psigmano2, s5ptime, s5pno2_units = load_s5p_data(df['path_s5p'][i], df['min_lat'][i],
                                                                               df['max_lat'][i], df['min_lon'][i], df['max_lon'][i])
    oco2lat, oco2lon, oco2co2, oco2sigmaco2, oco2time, oco2co2_units = load_oco2_data(df['path_oco2'][i], df['min_lat'][i],
                                                                                      df['max_lat'][i], df['min_lon'][i], df['max_lon'][i])
    era5lat, era5lon, era5u10, era5v10, era5u100, era5v100, era5time, era5tcc = load_era5_data(df['path_era5'][i])
    (era5_spread_lat, era5_spread_lon, era5_spread_u10, era5_spread_v10, era5_spread_u100,
     era5_spread_v100, era5_spread_time, era5_spread_tcc) = load_era5_data(df['path_spread_era5'][i])
    era5_idx = np.argmin(np.abs(era5time - oco2time[0]))
    era5lat, era5lon, era5u10, era5v10, era5u100, era5v100, era5time, era5tcc = (era5lat[era5_idx], era5lon[era5_idx], 
                                                                                 era5u10[era5_idx], era5v10[era5_idx],
                                                                                 era5u100[era5_idx], era5v100[era5_idx], 
                                                                                 era5time[era5_idx], era5tcc[era5_idx])
    (era5_spread_lat, era5_spread_lon, era5_spread_u10, era5_spread_v10, era5_spread_u100,
     era5_spread_v100, era5_spread_time, era5_spread_tcc) = (era5_spread_lat[0], era5_spread_lon[0], 
                                                             era5_spread_u10[0], era5_spread_v10[0],
                                                             era5_spread_u100[0], era5_spread_v100[0],
                                                             era5_spread_time[0], era5_spread_tcc[0])
    

    """Calculate wind speed and wind direction."""
    x1, y1, x2, y2, x3, y3, x4, y4 = bounding_box(oco2lat, oco2lon, k_numbers=data_points_width_k, buffer=buffer_bounding_box)
    origin = np.array([np.mean([x1,x2,x3,x4]), np.mean([y1,y2,y3,y4])])
    lat_origin, lon_origin = origin[1], origin[0]
    latbox, lonbox = [df['min_lat'][i], df['max_lat'][i]], [df['min_lon'][i], df['max_lon'][i]]
    lat_0, lon_0 = df['center_lat'][i], df['center_lon'][i]

    ws10m = np.sqrt(np.array(era5u10)**2 + np.array(era5v10)**2)
    wsspread10m = np.sqrt(np.array(era5_spread_u10)**2 + np.array(era5_spread_v10)**2)  # Pythagoras
    sigma_ws10m = np.mean(wsspread10m)  # Mean of the uncertainties in the wind direction
    xwindmat10m = np.linspace(df['min_lon'][i], df['max_lon'][i], era5u10.shape[1])  # Matrix of x-wind directions in selected area
    ywindmat10m = np.linspace(df['min_lat'][i], df['max_lat'][i], era5v10.shape[0])  # Matrix of y-wind directions in selected area
    xwindspreadmat10m = np.linspace(df['min_lon'][i], df['max_lon'][i], era5_spread_u10.shape[1])  # Uncertainty in the x-wind direction     
    ywindspreadmat10m = np.linspace(df['min_lat'][i], df['max_lat'][i], era5_spread_v10.shape[0])  # Uncertainty in the y-wind direction

    """Calculate wind direction and wind speed at the center of the bounding box."""
    distances10m = np.sqrt((xwindmat10m[:, np.newaxis] - lon_origin)**2 + (ywindmat10m[np.newaxis, :] - lat_origin)**2)  # Distance from origin
    distances_spread10m = np.sqrt((xwindspreadmat10m[:, np.newaxis] - lon_origin)**2 + (ywindspreadmat10m[np.newaxis, :] - lat_origin)**2)
    min_idx10m = np.unravel_index(np.argmin(distances10m), distances10m.shape)  # Index of the minimum distance to the origin
    min_idxspread10m = np.unravel_index(np.argmin(distances_spread10m), distances_spread10m.shape)  # Index of the minimum distance to the origin


    """Calculate wind direction and wind speed at the center of the bounding box."""
    uwind_center10m = era5u10[min_idx10m]
    vwind_center10m = era5v10[min_idx10m]
    uwind_spread_center10m = era5_spread_u10[min_idxspread10m]
    vwind_spread_center10m = era5_spread_v10[min_idxspread10m]
    center_wind_speed_10m = ws10m[min_idx10m]

    """The uncertainty in the wind direction is calculated using the formula for the standard deviation of a product of two variables."""
    x2y2_minus_one10m = 1/ np.sqrt(uwind_center10m**2 + vwind_center10m**2)  # Uncertainties on the angle theta 1/sqrt(x^2 + y^2)
    sigma_theta10m = np.sqrt( (vwind_center10m*x2y2_minus_one10m*uwind_spread_center10m)**2 +
                             (uwind_spread_center10m*x2y2_minus_one10m*vwind_spread_center10m)**2 )  # Uncertainty in the wind direction 
    
    """The same calculations are done for the 100m wind field."""
    ws100m = np.sqrt(np.array(era5u100)**2 + np.array(era5v100)**2)
    wsspread100m = np.sqrt(np.array(era5_spread_u100)**2 + np.array(era5_spread_v100)**2)
    sigma_ws100m = np.mean(wsspread100m)
    xwindmat100m = np.linspace(df['min_lon'][i], df['max_lon'][i], era5u100.shape[1])
    ywindmat100m = np.linspace(df['min_lat'][i], df['max_lat'][i], era5v100.shape[0])
    xwindspreadmat100m = np.linspace(df['min_lon'][i], df['max_lon'][i], era5_spread_u100.shape[1])
    ywindspreadmat100m = np.linspace(df['min_lat'][i], df['max_lat'][i], era5_spread_v100.shape[0])
    distances100m = np.sqrt((xwindmat100m[:, np.newaxis] - lon_origin)**2 + (ywindmat100m[np.newaxis, :] - lat_origin)**2)
    distances_spread100m = np.sqrt((xwindspreadmat100m[:, np.newaxis] - lon_origin)**2 + (ywindspreadmat100m[np.newaxis, :] - lat_origin)**2)
    min_idx100m = np.unravel_index(np.argmin(distances100m), distances100m.shape)
    min_idxspread100m = np.unravel_index(np.argmin(distances_spread100m), distances_spread100m.shape)
    uwind_center100m = era5u100[min_idx100m]
    vwind_center100m = era5v100[min_idx100m]
    uwind_spread_center100m = era5_spread_u100[min_idxspread100m]
    vwind_spread_center100m = era5_spread_v100[min_idxspread100m]
    center_wind_speed_100m = ws100m[min_idx100m]
    x2y2_minus_one100m = 1/ np.sqrt(uwind_center100m**2 + vwind_center100m**2) # Uncertainties on the angle theta 1/sqrt(x^2 + y^2)
    sigma_theta100m = np.sqrt( (vwind_center100m*x2y2_minus_one100m*uwind_spread_center100m)**2 +
                             (uwind_spread_center100m*x2y2_minus_one100m*vwind_spread_center100m)**2 )
    
    """Calculate string for wind both speed plot."""
    str_10m_wind, str_100m_wind = stringcalc(center_wind_speed_10m, sigma_ws10m, center_wind_speed_100m, sigma_ws100m)


    """Plotting"""
    fig, ((ax1, ax2), (ax4, ax6)) = plt.subplots(2,2, figsize=(20,20))

    """Name of the location of the i'th plot."""
    namei = df['name'][i]
    fig.suptitle(f'{namei}', fontsize=16)

    """First plot"""
    m = make_basemap(ax1, latbox, lonbox, lat_0, lon_0, addlines=False)
    if s5plat.shape[0] != 0:  # Check if there is any data
        s5ptimerounded = s5ptime[0].round('s').tz_localize(None).strftime('%d-%m-%Y %H:%M:%S')
        ax1.set_title(f'S5P NO$_{2}$ at {s5ptimerounded}')
    plotno2_func(ax1, m, s5plon, s5plat, lat_origin, lon_origin, 
                 s5pno2, s5pno2_units, uwind_center10m, vwind_center10m, vminno2, vmaxno2, str_10m_wind)
    
    """Plot bounding box"""
    basemap_x1, basemap_y1 = m(x1, y1)
    basemap_x2, basemap_y2 = m(x2, y2)
    basemap_x3, basemap_y3 = m(x3, y3)
    basemap_x4, basemap_y4 = m(x4, y4)

    ax1.plot([basemap_x1, basemap_x2, basemap_x3, basemap_x4, basemap_x1], 
             [basemap_y1, basemap_y2, basemap_y3, basemap_y4, basemap_y1], 'k', label='Selected data bounding box')
    ax1.legend(loc = 'upper right')

    """Second plot"""
    m = make_basemap(ax2, latbox, lonbox, lat_0, lon_0, addlines=False)
    if oco2lat.shape[0] != 0:
        oco2timerounded = oco2time[0].round('s').tz_localize(None).strftime('%d-%m-%Y %H:%M:%S')
        ax2.set_title(f'OCO2 CO$_{2}$ at {oco2timerounded}')
        # print(oco2timerounded)
    if oco2lat.shape[0] != 0:
        plotco2_func(ax2, m, oco2lon, oco2lat, lat_origin, lon_origin, oco2co2, oco2co2_units,
                     uwind_center10m, vwind_center10m, vminco2, vmaxco2, str_10m_wind)

    ax2.plot([basemap_x1, basemap_x2, basemap_x3, basemap_x4, basemap_x1], 
             [basemap_y1, basemap_y2, basemap_y3, basemap_y4, basemap_y1], 'k', label='Selected data bounding box')
    ax2.legend(loc = 'upper right')
    
    """Wind field plot"""
    # """Fhird plot"""
    # m = make_basemap(ax3, latbox, lonbox, lat_0, lon_0, addlines=False)
    # if era5lat.shape != 0:
    #     era5timerounded = era5time.round('s').tz_localize(None)
    #     ax3.set_title(f'ERA5 10m wind at {era5timerounded}')
    # plotwind_func(ax3, m, xwindmat10m, ywindmat10m, era5u10, era5v10, ws10m, str_10m_wind)
    # ax3.legend(loc = 'upper right')

    # m = make_basemap(ax7, latbox, lonbox, lat_0, lon_0, addlines=False)
    # if era5lat.shape != 0:
    #     era5timerounded = era5time.round('s').tz_localize(None)
    #     ax7.set_title(f'ERA5 100m wind at {era5timerounded}')
    # plotwind_func(ax7, m, xwindmat10m, ywindmat10m, era5u100, era5v100, ws100m, str_100m_wind)
    # ax7.legend(loc = 'upper right')

    # m = make_basemap(ax8, latbox, lonbox, lat_0, lon_0, addlines=False)
    # if era5lat.shape != 0:
    #     era5timerounded = era5time.round('s').tz_localize(None)
    #     ax8.set_title(f'ERA5 100m spread')
    # plotwind_func(ax8, m, xwindspreadmat10m, ywindspreadmat10m, era5_spread_u10, era5_spread_v10, wsspread10m, str_10m_wind)
    # ax8.legend(loc = 'upper right')

    """Fourth plot"""
    polygon = Polygon(zip(np.array([x1, x2, x3, x4]), np.array([y1, y2, y3, y4])))
    points_within_polygon = [Point(x, y, z) for x, y, z, _ in zip(s5plon, s5plat, s5pno2, s5psigmano2) if Point(x, y, z).within(polygon)]
    points_within_polygon_array = np.array([(point.x, point.y, point.z, otherparam) 
                                            for point, otherparam in zip(points_within_polygon, s5psigmano2) if point.within(polygon)])
    s5plat_box, s5plon_box, s5pno2_box, s5psigmano2_box = (points_within_polygon_array[:,1], points_within_polygon_array[:,0], 
                                                           points_within_polygon_array[:,2], points_within_polygon_array[:,3])

    """Select units factor of NO2"""
    s5pno2_box = s5pno2_box*1e3  # Select units factor of NO2
    s5psigmano2_box = s5psigmano2_box*1e3  # Select units factor of NO2

    """Calculate distance between points"""
    distance_s5p = np.array([haversine([s5plat_box[0], s5plon_box[0]], [s5plat_box[j], s5plon_box[j]]) for j in range(len(s5plat_box))])
    distance_oco2 = np.array([haversine([oco2lat[0], oco2lon[0]], [oco2lat[i], oco2lon[i]]) for i in range(len(oco2lat))])

    """Fit the data"""
    initial_guess_no2 = ini_guess(distance_s5p, s5pno2_box)
    initial_guess_co2 = ini_guess(distance_oco2, oco2co2)
    (a0, a1, a2, a3, la4), covariance_no2 = curve_fit(model_no2, distance_s5p, s5pno2_box, p0=initial_guess_no2)
    (a5, a6, a7, a8), covariance_co2 = curve_fit(wrapperfunc(la4), distance_oco2, oco2co2, p0=initial_guess_co2[:-1])

    """Calculate the errors using the covariance matrix and the differention 
    error propagation formula then multiply by a4 on each side of the equation""" 
    s5p_err_no2 = np.sqrt(np.diag(covariance_no2))
    s5p_err_co2 = np.sqrt(np.diag(covariance_co2))
    s5p_err_no2[-1] = np.exp(la4) * s5p_err_no2[-1]  # Multiply by a4 on the last parameter to get the error on a4 from la4

    """Add a4 to the list of parameters"""
    new_params_no2 = a0, a1, a2, a3, np.exp(la4)  # Add a4 to the list of parameters instead of la4
    new_params_co2 = a5, a6, a7, a8, np.exp(la4)  # Add a4 to the list of parameters instead of la4

    """Plot the data and the fit"""
    xx = np.linspace(0, np.max(distance_oco2), 1000)
    ax4.set_title(f'CO$_{2}$ and NO$_{2}$ vs Distance for {namei}')
    ax4.plot(distance_oco2, oco2co2, '.', color='r')
    ax4.plot(distance_oco2, oco2co2, '.', color='r', zorder=4)
    ax4.plot(xx, non_log_model_co2(xx, *new_params_co2), '--', color='k', label='XCO2', zorder=3)
    ax4.set_xlabel('Distance [km]')
    ax4.set_ylabel(f'XCO2 [{oco2co2_units}]', color='r')
    ax4.set_ylim([a5-3*a7, a5+3*a7])
    ax4.yaxis.tick_left()
    ax4.legend(loc=2)
    
    ax4_2 = ax4.twinx() 
    ax4_2.plot(distance_s5p, s5pno2_box, '.', color='k')
    ax4_2.plot(distance_s5p, s5pno2_box, '.', color='k', zorder=5)
    ax4_2.plot(xx, non_log_model_no2(xx, *new_params_no2), '-', color='k', label='NO$_2$', zorder=2)
    ax4_2.set_ylabel(r'NO$_2$ [$\frac{mmol}{m^{2}}$]', color='k')
    ax4_2.set_ylim([a0-3*a2, a0+3*a2])
    ax4_2.yaxis.tick_right()
    ax4_2.legend(loc=1)



    """Fifth plot"""
    """Calculate the angle between the wind vector and the satellite swath path"""
    vector_wind_10m = np.array([uwind_center10m, vwind_center10m])
    vector_wind_100m = np.array([uwind_center100m, vwind_center100m])

    vector_data = np.array([(np.mean([x1,x2])-np.mean([x3,x4])), (np.mean([y1,y2])-np.mean([y3,y4]))])


    normal_vector = np.array([-vector_data[1], vector_data[0]])  # Non-normalised normal vector to the satellite swath path
    if np.cross(vector_wind_10m, vector_data) > 0:
        normal_vector *= -1  # Make sure the normal vector points in the right direction

    if np.cross(vector_wind_100m, vector_data) > 0:
        normal_vector *= -1  # Make sure the normal vector points in the right direction
    
    normal_vector_normalized = normal_vector / np.linalg.norm(normal_vector)  # Normalize the normal vector

    
    """Below should have been a function and used for both 10m and 100m but this is okay for now."""
    theta_rad_10m = np.arccos(np.dot(vector_wind_10m, vector_data) / (np.linalg.norm(vector_wind_10m) * np.linalg.norm(vector_data)))
    theta_rad_10m = np.arccos(np.dot(vector_wind_100m, vector_data) / (np.linalg.norm(vector_wind_100m) * np.linalg.norm(vector_data)))
    theta_deg_10m = np.degrees(theta_rad_10m)  # Angle between the wind vector and the satellite swath path
    theta_deg_100m = np.degrees(theta_rad_10m)  # Angle between the wind vector and the satellite swath path

    if theta_deg_10m > 90.0:
        theta_min_deg_10m = 180.0 - theta_deg_10m
    else:
        theta_min_deg_10m = theta_deg_10m
    if theta_deg_100m > 90.0:
        theta_min_deg_100m = 180.0 - theta_deg_100m
    else:
        theta_min_deg_100m = theta_deg_100m

    sin_theta_min_10m = np.sin(np.radians(theta_min_deg_10m))
    sin_theta_min_100m = np.sin(np.radians(theta_min_deg_100m))
    desired_length_10m = np.linalg.norm(vector_wind_10m) * sin_theta_min_10m  # Desired length of the normal vector to the satellite swath path
    desired_length_100m = np.linalg.norm(vector_wind_100m) * sin_theta_min_100m  # Desired length of the normal vector to the satellite swath path
    final_vector_10m = desired_length_10m * normal_vector_normalized  # Final vector with the desired length and direction
    final_vector_100m = desired_length_100m * normal_vector_normalized  # Final vector with the desired length and direction


    """Normal wind vector plot"""
    # ax5.plot([x1, x2, x3, x4, x1], [y1, y2, y3, y4, y1], 'k', label='Satellite swath path boundary box')
    # ax5.quiver(*([x3 - (x3-x4)/2, y3]), *vector_data, angles='xy', scale_units='xy', scale=1, color='red', label='Satellite swath path')
    # ax5.quiver(*origin, *vector_wind_10m, angles='xy', scale_units='xy', scale=7, color='blue', label='Vector Wind')
    # ax5.quiver(*origin, *(final_vector), angles='xy', scale_units='xy', scale=7, color='green', label='Normal Wind')
    # ax5.set_title(f'{namei}\n' + f'Angle: {theta_deg_10m:.2f} degrees\nSmallest Angle: {theta_min_deg:.2f} degrees')
    # ax5.set_aspect('equal', 'box')
    # ax5.legend(loc='upper right')

    """Sixth plot"""
    """Residual plot"""
    n_for_sigma = 3  # Number of standard deviations used for DOF calculation
    nsigma_no2 = n_for_sigma*new_params_no2[4]/(2*np.sqrt(2*np.log(2)))
    nsigma_co2 = n_for_sigma*new_params_co2[4]/(2*np.sqrt(2*np.log(2)))
    range_idx_no2 = np.argwhere((new_params_no2[3]-nsigma_no2<distance_s5p) & (distance_s5p<new_params_no2[3]+nsigma_no2))
    range_idx_co2 = np.argwhere((new_params_co2[3]-nsigma_co2<distance_oco2) & (distance_oco2<new_params_co2[3]+nsigma_co2))
    DOF_no2 = len(range_idx_no2)-len(new_params_no2)  # Degrees of freedom for NO2
    DOF_co2 = len(range_idx_co2)-len(new_params_co2)  # Degrees of freedom for CO2

    y_expected_no2 = non_log_model_no2(distance_s5p, *new_params_no2)  # Expected NO2 values
    y_expected_co2 = non_log_model_co2(distance_oco2, *new_params_co2)  # Expected CO2 values

    residuals_no2 = s5pno2_box-y_expected_no2  # Residuals for NO2
    residuals_co2 = oco2co2-y_expected_co2  # Residuals for CO2

    chi2_no2 = np.sum(residuals_no2**2/y_expected_no2)  # Chi-square for NO2
    chi2_co2 = np.sum(residuals_co2**2/y_expected_co2)  # Chi-square for CO2

    p_no2 = 1 - sp.stats.chi2.cdf(chi2_no2, DOF_no2)  # p-value for NO2
    p_co2 = 1 - sp.stats.chi2.cdf(chi2_co2, DOF_co2)  # p-value for CO2

    """Plotting the residuals"""
    ax6.errorbar(distance_oco2, residuals_co2, yerr=oco2sigmaco2, color='r', 
                 label=f'XCO2: Chi-square: {chi2_co2:.3f}: p-value: {p_co2:.5e}', fmt='.', lw=0.5, zorder=3)
    ax6.plot(xx, np.zeros(len(xx)), 'k--', zorder=1)
    ax6.set_title(f'Residual of CO$_{2}$ and NO$_{2}$ vs Distance for {namei}')
    ax6.set_xlabel('Distance [km]')
    ax6.set_ylabel(f'XCO2 [{oco2co2_units}]', color='r')
    ax6.set_ylim([0-8*a7, 0+4*a7])
    ax6.yaxis.tick_left()
    ax6_2 = ax6.twinx()
    ax6_2.errorbar(distance_s5p, residuals_no2, yerr=s5psigmano2_box, color='k', 
                   label=f'NO2: Chi-square: {chi2_no2:.3f}: p-value: {p_no2:.5e}', fmt='.', zorder=1)
    ax6_2.set_ylabel(r'NO$_2$ [$\frac{mmol}{m^{2}}$]', color='k')
    ax6_2.set_ylim([0-8*a2, 0+4*a2])
    ax6_2.yaxis.tick_right()

    lines6, labels6 = ax6.get_legend_handles_labels()
    lines6_2, labels6_2 = ax6_2.get_legend_handles_labels()
    ax6_2.legend(lines6 + lines6_2, labels6 + labels6_2, loc=1)

    axins = inset_axes(ax6, width='35%', height='35%', loc='lower left' , borderpad=4)
    axins.hist(residuals_co2, histtype='step', lw=2, color='r')
    axins.set(xlabel=f'Histogram XCO2 [{oco2co2_units}]', ylabel='Counts')
    axins.patch.set_alpha(0.5)

    axins_2 = inset_axes(ax6_2, width='35%', height='35%', loc='lower right' , borderpad=4)
    axins_2.hist(residuals_no2, histtype='step', lw=2, color='k')
    axins_2.set(xlabel= r'Histogram NO$_2$ [$\frac{mmol}{m^{2}}$]', ylabel='')
    axins_2.patch.set_alpha(0.5)

    testno2 = np.abs(residuals_no2)-s5psigmano2_box
    testco2 = np.abs(residuals_co2)-oco2sigmaco2

    plt.tight_layout()
    """Save the plot and show the plot."""
    plt.savefig(f'./Result_{namei}.png', dpi=600)
    plt.show()


    """Save data to lists for calculating mass flow rate."""
    origin_list.append(origin)
    sin_theta_min_list10m.append(sin_theta_min_10m)
    desired_length_list10m.append(desired_length_10m)
    final_vector_list10m.append(final_vector_10m)
    theta_min_deg_list10m.append(theta_min_deg_10m)

    sin_theta_min_list100m.append(sin_theta_min_100m)
    desired_length_list100m.append(desired_length_100m)
    final_vector_list100m.append(final_vector_100m)
    theta_min_deg_list100m.append(theta_min_deg_100m)
    s5p_err_no2_list.append(s5p_err_no2)
    s5p_err_co2_list.append(s5p_err_co2)
    sigma_ws10m_list.append(sigma_ws10m)
    sigma_ws100m_list.append(sigma_ws100m)
    sigma_theta10m_list.append(sigma_theta10m)
    sigma_theta100m_list.append(sigma_theta100m)
    new_params_no2_list.append(new_params_no2)
    new_params_co2_list.append(new_params_co2)

In [ ]:
def kg_per_sec_to_mt_per_year(kg_per_sec):
    sec_to_year = 60 * 60 * 24 * 365.25
    kg_to_mt = 1 / (10**6)
    mt_per_year = kg_per_sec * sec_to_year * kg_to_mt
    return mt_per_year

In [ ]:
"""Result from the article"""
test = 1/2*np.sqrt(np.pi/np.log(2)) * (0.04401 / 6.02214e23) * 2.15e29 * 1e-6  # This is where 0.53 comes from
print(kg_per_sec_to_mt_per_year(test))

In [ ]:
origin_list = np.array(origin_list)
sin_theta_min_list10m = np.array(sin_theta_min_list10m)
desired_length_list10m = np.array(desired_length_list10m)
final_vector_list10m = np.array(final_vector_list10m)
theta_min_deg_list10m = np.array(theta_min_deg_list10m)

sin_theta_min_list100m = np.array(sin_theta_min_list100m)
desired_length_list100m = np.array(desired_length_list100m)
final_vector_list100m = np.array(final_vector_list100m)
theta_min_deg_list100m = np.array(theta_min_deg_list100m)

s5p_err_no2_list = np.array(s5p_err_no2_list)
s5p_err_co2_list = np.array(s5p_err_co2_list)
sigma_ws10m_list = np.array(sigma_ws10m_list)
sigma_ws100m_list = np.array(sigma_ws100m_list)
sigma_theta10m_list = np.array(sigma_theta10m_list)
sigma_theta100m_list = np.array(sigma_theta100m_list)
new_params_no2_list = np.array(new_params_no2_list)
new_params_co2_list = np.array(new_params_co2_list)

a0, a1, a2, a3, a4 = new_params_no2_list[:,0], new_params_no2_list[:,1], new_params_no2_list[:,2], new_params_no2_list[:,3], new_params_no2_list[:,4]
a5, a6, a7, a8 = new_params_co2_list[:,0], new_params_co2_list[:,1], new_params_co2_list[:,2], new_params_co2_list[:,3]

# desired_length_list_article = np.array([1.6, 0.9, 4.4, 2.6, 6.7])  # The values from the article - only for testing

In [ ]:
def result_string(result, sigma_result):
    """Return result string with uncertainty."""
    u_res = ufloat(result, sigma_result)
    return f"{u_res:.2u}"

In [ ]:
"""Mass flow rate estimation for 10m wind"""

for i in range(5):
    print(df['name'][i])
    # print(f'a4 width FWHM of the gauss is {new_params_no2_list[i,4]:.2f} km')
    # print('a7 the amplitude of the CO2 gauss is', new_params_co2_list[i,2])
    dot_m_delta_CO2 =  1/2*np.sqrt(np.pi/np.log(2)) * (0.04401 / 6.02214e23) * 2.15e29 * new_params_no2_list[i,4] * 1e-6*new_params_co2_list[i,2] * desired_length_list10m[i]

    a4 = new_params_no2_list[i,4]
    a7 = new_params_co2_list[i,2]

    a4_sigma = s5p_err_no2_list[i,4]
    a7_sigma = s5p_err_co2_list[i,2]

    v = desired_length_list10m[i]
    v_sigma = sigma_ws10m_list[i]

    theta = theta_min_deg_list10m[i]
    theta_sigma = sigma_theta10m_list[i]    

    k = 1/2*np.sqrt(np.pi/np.log(2)) * (0.04401 / 6.02214e23) * 2.15e29 * 1e-6
    mass_flow_rate_sigma = k*np.sqrt((a7*v*np.sin(np.deg2rad(theta))*a4_sigma)**2  +  
                                     (a4*v*np.sin(np.deg2rad(theta))*a7_sigma)**2  +  
                                     (a4*a7*np.sin(np.deg2rad(theta))*v_sigma)**2  +  
                                     (-a4*a7*v*np.cos(np.deg2rad(theta))*theta_sigma)**2)

    test1 = np.exp(k)*np.sqrt((a7*v*np.sin(np.deg2rad(theta))*a4_sigma)**2)
    test2 = np.exp(k)*np.sqrt((a4*v*np.sin(np.deg2rad(theta))*a7_sigma)**2)
    test3 = np.exp(k)*np.sqrt((a4*a7*np.sin(np.deg2rad(theta))*v_sigma)**2)
    test4 = np.exp(k)*np.sqrt((-a4*a7*v*np.cos(np.deg2rad(theta))*theta_sigma)**2)
    # print(f'First term in propergation a4 is: {test1:.2f}, sigma is {a4_sigma:.2f} km,')
    # print(f'Second term in propergation a7 is: {test2:.2f}, sigma is {a7_sigma:.2f} ppm,')
    # print(f'Third term in propergation v is: {test3:.2f}, sigma is {v_sigma:.2f} m/s,')
    # print(f'Fourth term in propergation theta is: {test4:.2f}, sigma is {theta_sigma:.5f} rad,')
    # print('v is', result_string(v, v_sigma))
    # print('theta is', result_string(theta, theta_sigma))
    # print('width FWHM of the gauss is', result_string(a4, a4_sigma))
    # print('amplitude of the CO2 gauss is', result_string(a7, a7_sigma))
    print(f'Mass flow rate is:', result_string(kg_per_sec_to_mt_per_year(dot_m_delta_CO2), kg_per_sec_to_mt_per_year(mass_flow_rate_sigma)) )
    # print(f'Mass flow rate is:', kg_per_sec_to_mt_per_year(dot_m_delta_CO2), 'with sigma:', kg_per_sec_to_mt_per_year(mass_flow_rate_sigma))
    print('')

In [ ]:
"""Mass flow rate estimation for 100m wind"""

for i in range(5):
    print(df['name'][i])
    # print(f'a4 width FWHM of the gauss is {new_params_no2_list[i,4]:.2f} km')
    # print('a7 the amplitude of the CO2 gauss is', new_params_co2_list[i,2])
    dot_m_delta_CO2 =  1/2*np.sqrt(np.pi/np.log(2)) * (0.04401 / 6.02214e23) * 2.15e29 * new_params_no2_list[i,4] * 1e-6*new_params_co2_list[i,2] * desired_length_list100m[i]

    a4 = new_params_no2_list[i,4]
    a7 = new_params_co2_list[i,2]

    a4_sigma = s5p_err_no2_list[i,4]
    a7_sigma = s5p_err_co2_list[i,2]

    v = desired_length_list100m[i]
    v_sigma = sigma_ws100m_list[i]

    theta = theta_min_deg_list100m[i]
    theta_sigma = sigma_theta100m_list[i]    

    k = 1/2*np.sqrt(np.pi/np.log(2)) * (0.04401 / 6.02214e23) * 2.15e29 * 1e-6
    mass_flow_rate_sigma = k*np.sqrt((a7*v*np.sin(np.deg2rad(theta))*a4_sigma)**2  +  
                                     (a4*v*np.sin(np.deg2rad(theta))*a7_sigma)**2  +  
                                     (a4*a7*np.sin(np.deg2rad(theta))*v_sigma)**2  +  
                                     (-a4*a7*v*np.cos(np.deg2rad(theta))*theta_sigma)**2)

    test1 = np.exp(k)*np.sqrt((a7*v*np.sin(np.deg2rad(theta))*a4_sigma)**2)
    test2 = np.exp(k)*np.sqrt((a4*v*np.sin(np.deg2rad(theta))*a7_sigma)**2)
    test3 = np.exp(k)*np.sqrt((a4*a7*np.sin(np.deg2rad(theta))*v_sigma)**2)
    test4 = np.exp(k)*np.sqrt((-a4*a7*v*np.cos(np.deg2rad(theta))*theta_sigma)**2)
    # print(f'First term in propergation a4 is: {test1:.2f}, sigma is {a4_sigma:.2f} km,')
    # print(f'Second term in propergation a7 is: {test2:.2f}, sigma is {a7_sigma:.2f} ppm,')
    # print(f'Third term in propergation v is: {test3:.2f}, sigma is {v_sigma:.2f} m/s,')
    # print(f'Fourth term in propergation theta is: {test4:.2f}, sigma is {theta_sigma:.5f} rad,')
    print('v is', result_string(v, v_sigma))
    print('theta is', result_string(theta, theta_sigma))
    print('width FWHM of the gauss is', result_string(a4, a4_sigma))
    print('amplitude of the CO2 gauss is', result_string(a7, a7_sigma))
    print('Mass flow rate is', result_string(kg_per_sec_to_mt_per_year(dot_m_delta_CO2), kg_per_sec_to_mt_per_year(mass_flow_rate_sigma)) )
    # print(f'Mass flow rate is:', kg_per_sec_to_mt_per_year(dot_m_delta_CO2), 'with sigma:', kg_per_sec_to_mt_per_year(mass_flow_rate_sigma))
    print('')